# Belief Revision Assignment

02180 Introduction to Artificial Intelligence

You are asked to implement a belief revision agent. By default, the project should run through
the following sequence of stages:
1. design and implementation of belief base;
2. design and implementation of a method for checking logical entailment (e.g., resolution-based), you should implement it yourself, without using any existing packages;
3. implementation of contraction of belief base (based on a priority order on alphas in the belief base);
4. implementation of expansion of belief base.
The output should be the resulting/new belief base.

## TODO

- [ ] Method to check logical entailment
- [ ] Contraction of belief base
- [ ] Expansion of belief base
- [ ] Use AGM postulates (lecture 11) to test your algorithm

In [1]:
from sympy.logic.boolalg import (
    to_cnf,
    Not,
    And,
    Or,
    Implies,
    Equivalent,
)
# from sympy import Equality

In [2]:
def disjuncts(clause) -> list:
    """
    Separate the clause by the Or operator.

    Returns:
        list: a list of disjuncts in the given clause.

    Adapted from https://github.com/tdiam/belief-revision-engine

    Usage:
        disjuncts(A | B | C) -> [A, B, C]
        disjuncts(A | (B & C)) -> [A, B & C]
    """
    return dissociate(Or, [clause])


def conjuncts(clause) -> list:
    """
    Separate the clause by the And operator.

    Returns:
        list: a list of conjuncts in the given clause.

    Adapted from https://github.com/tdiam/belief-revision-engine

    Usage:
        conjuncts(A & B & C) -> [A, B, C]
    """
    return dissociate(And, [clause])


def dissociate(op, args: list) -> list:
    """
    Separate the arguments of a clause by the given operator.

    Adapted from https://github.com/tdiam/belief-revision-engine

    Args:
        op (And, Or): the SymPy operator to separate the arguments by
        args (list): the arguments to dissociate

    Returns:
        list: a list of arguments in the given clause

    Usage:
        dissociate(Or, [A | B | C]) -> [A, B, C]
        dissociate(And, [A & B & C]) -> [A, B, C]
        dissociate(Or, [A | B & C]) -> [A, B & C]
    """
    result = []

    def collect(subargs):
        for arg in subargs:
            if isinstance(arg, op):
                collect(arg.args)
            else:
                result.append(arg)

    collect(args)
    return result


def make_clause(args):
    """
    Create a disjunction of clauses from a list of arguments.

    Usage:
        make_clause([A, B, C]) -> A | B | C
        make_clause([A, B & C]) -> A | (B & C)
    """
    args = dissociate(Or, args)
    if len(args) == 0:  # there are no arguments
        return False
    elif len(args) == 1:
        return args[0]
    else:
        return Or(*args)

In [43]:
# Demonstration of the functions above
to_cnf("A | B & C")

(A | B) & (A | C)

In [22]:
conjuncts(to_cnf("(A & B) | C"))

[A | C, B | C]

In [17]:
disjuncts(to_cnf("A | B | C"))

[A, B, C]

In [5]:
dissociate(Or, [to_cnf("A | B | C")])

[A, B, C]

In [37]:
dissociate(And, [to_cnf("A | B & C")])

[A | B, A | C]

In [41]:
make_clause(to_cnf(["A | B & C"]))

A | (B & C)

In [7]:
class BeliefBase:
    def __init__(self) -> None:
        self.beliefs = []
        self.belief_base = []
        self.operations = set([Not, And, Or, Implies, Equivalent])

    # def add_belief(self, belief) -> None:
    #     """Add a belief to the belief base, keeping the belief base in CNF form"""
    #     belief = to_cnf(belief)
    #     #if belief.is_Atom:
    #      #   belief = to_cnf(And(belief, belief))
    #     self.belief_base.append(belief)

    def add_belief(self, belief: str) -> None:
        """Beliefs are added as CNF formulas."""
        self.beliefs.append(belief)

        # Eliminate bi-implications, assume one bi-imp per belief
        ## Find the bi-imp operator
        imp_op = None
        belief_s = belief.split()
        for i in range(len(belief_s)):
            if belief_s[i] == "<>":
                a = " ".join(belief_s[:i])
                b = " ".join(belief_s[i + 1 :])
                imp_op = belief_s[i]
                break
        ## Add the beliefs to the belief base
        if imp_op is None:  # No bi-imp op found
            self.belief_base.append(to_cnf(belief))
        elif imp_op == "<>":
            imply1 = to_cnf(f"{a} >> {b}")
            imply2 = to_cnf(f"{b} >> {a}")
            self.belief_base.extend([imply1, imply2])
        else:
            raise ValueError("Invalid implication operator")

    def pl_resolution(self, alpha) -> bool:
        """
        Implements the pl-resolution algorithm for clauses resolution
        to check for entailment.

        Args:
            alpha (sympy.Expr): A SymPy sentence representing the query.

        Returns:
            bool: True if the query is entailed by the knowledge base, False otherwise.
        """
        # Split each clause in the base by the "And" symbol
        clauses = []  # KB as list of disjunctions (CNF form)
        for clause in self.belief_base:
            clauses += conjuncts(clause)

        # Convert KB & ~alpha to CNF
        alpha_lit = conjuncts(to_cnf(Not(alpha)))  # Add negation of alpha as clause
        for clause in alpha_lit:
            clauses.append(clause)

        # Apply resolution rule to resulting clauses
        new_clauses = set()
        while True:
            resolvents = set()
            for ci in clauses:
                for cj in clauses:
                    if ci != cj:
                        resolvents |= self.pl_resolve(ci, cj)
                if any(clause is False for clause in resolvents):  # Empty clause found
                    return True

            if resolvents.issubset(clauses):  # No new clauses derived
                return False

            # Add new clauses to KB
            new_clauses |= resolvents
            clauses.extend(new_clauses)

    def pl_resolve(self, clause1, clause2) -> set:
        """
        Performs the full resolution of two clauses.

        Args:
            clause1 (sympy.Expr): A SymPy sentence representing a clause.
            clause2 (sympy.Expr): A SymPy sentence representing a clause.

        Returns:
            set: a set of resulting clauses from resolution.

        Usage:
            pl_resolve(A | B | C | ~D, ~A | D | E | F) -> {B | C | D | E | F}
        """
        clause1 = disjuncts(clause1)
        clause2 = disjuncts(clause2)
        resolvents = set()

        for ci in clause1:
            for cj in clause2:
                if ci == Not(cj) or cj == Not(ci):
                    # remove ci and cj from resolved clause
                    res = (set(clause1) - {ci}).union(set(clause2) - {cj})
                    resolvents.add(make_clause(res))
        return resolvents

    def __str__(self) -> str:
        return str(self.belief_base)

| Operation | Symbol |
|:----- | :----- |
| Bi-implication | `<>` |
| Implication | `>>` or `<<` |
| Conjunction | `&` |
| Disjunction | `\|` |
| Negation | `~` |

In [10]:
# Create an instance of BeliefBase
kb = BeliefBase()

# Add to the belief base
kb.add_belief("q & p & ~r")
kb.add_belief("r | s")
print(f"After adding: {kb}")

# Check if a belief is entailed by the belief base
alpha = "q & ~r"
entails = kb.pl_resolution(alpha)
print(f"entails {alpha}: {entails}")

# The problem is that we have to feed the resolution function with a disjunction form and a sentence
# We also have to adapt the transformation functions to how our belief base works

After adding: [p & q & ~r, r | s]
entails q & ~r: True


In [9]:
# Test of add_belief() with bi-implication
kb = BeliefBase()

kb.add_belief("q & p & ~r")
print(f"After adding: {kb}")

kb.add_belief("r <> p | s")
print(f"After adding: {kb}")

After adding: [p & q & ~r]
After adding: [p & q & ~r, p | s | ~r, p | r | ~s]
